In [1]:
library(survey)
library(tidyverse)

Loading required package: grid
Loading required package: Matrix
Loading required package: survival

Attaching package: ‘survey’

The following object is masked from ‘package:graphics’:

    dotchart

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.3.0
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps_subgroup_vaso.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_pco2_flag,lab_creatinine_flag,lab_potassium_flag,lab_troponin_flag,lab_po2_flag,lab_lactate_flag,echo_int,mort_28_day_int,ps,ps_weight
214954,170515,54642,MICU,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,1,2176-03-11,⋯,1,1,1,1,1,1,1,0,0.7339584,1.362475
264413,134244,81436,MICU,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,1,2133-03-31,⋯,0,1,1,0,0,1,1,0,0.3889232,2.571202
214106,123152,74869,MICU,2150-09-05 17:50:32,2150-09-12 21:40:59,1,74.13371,1,2150-09-07,⋯,1,1,1,1,1,1,1,1,0.7721262,1.295125
254478,128652,31,MICU,2108-08-22 23:28:42,2108-08-30 21:59:20,1,72.26709,1,NA,⋯,1,1,1,0,1,1,0,1,0.1951075,1.242402
264446,165520,10013,MICU,2125-10-04 23:38:00,2125-10-07 15:13:52,1,87.08742,1,NA,⋯,1,1,1,0,1,0,0,1,0.4902299,1.961669
218143,122936,10370,SICU,2145-01-15 03:36:30,2145-01-20 19:00:30,1,45.06368,1,2145-01-14,⋯,1,1,1,0,1,1,1,0,0.8799575,1.136419


In [4]:
wtd_pval <- data.table::fread(file.path(data_dir, "wtd_pval_subgroup_vaso.csv"), data.table = FALSE)
head(wtd_pval)

cov,pval
age,0.026
gender,0.328
first_careunit,0.834
weight,0.122
saps,0.755
sofa,0.145


In [5]:
wpv <- wtd_pval %>%
    filter(pval < 0.05) %>%
    pull(cov)

wpv

[1] "age"                        "vent"                      
 [3] "sedative"                   "icd_chf"                   
 [5] "icd_afib"                   "icd_cad"                   
 [7] "icd_stroke"                 "vs_cvp_first"              
 [9] "lab_lactate_first"          "lab_creatinine_first"      
[11] "lab_bnp_flag"               "lab_troponin_flag"         
[13] "lab_creatinine_kinase_flag"

In [6]:
ipw_svydesign <- readRDS(file.path(data_dir, "ipw_svydesign_subgroup_vaso.rds"))

In [7]:
fml <- wpv %>%
c("echo", .) %>%
paste(collapse = " + ") %>%
sprintf("mort_28_day ~ %s", .)

fml

[1] "mort_28_day ~ echo + age + vent + sedative + icd_chf + icd_afib + icd_cad + icd_stroke + vs_cvp_first + lab_lactate_first + lab_creatinine_first + lab_bnp_flag + lab_troponin_flag + lab_creatinine_kinase_flag"

In [8]:
logi <- svyglm(as.formula(fml), family = quasibinomial, design = ipw_svydesign)

In [9]:
summary(logi)


Call:
svyglm(formula = as.formula(fml), family = quasibinomial, design = ipw_svydesign)

Survey design:
svydesign(ids = ~icustay_id, weights = ~ps_weight, data = full_data)

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 -3.495515   0.398304  -8.776  < 2e-16 ***
echo1                       -0.476362   0.147636  -3.227  0.00129 ** 
age                          0.022135   0.004726   4.684 3.12e-06 ***
vent1                        0.738897   0.273536   2.701  0.00700 ** 
sedative1                    0.060931   0.245138   0.249  0.80374    
icd_chf1                    -0.193666   0.170004  -1.139  0.25484    
icd_afib1                    0.219342   0.159839   1.372  0.17023    
icd_cad1                    -0.353070   0.205039  -1.722  0.08532 .  
icd_stroke1                  0.483422   0.313863   1.540  0.12376    
vs_cvp_first                 0.002408   0.003176   0.758  0.44836    
lab_lactate_first            0.221657   0

In [10]:
exp(cbind(OR = coef(logi), confint(logi)))

,OR,2.5 %,97.5 %
(Intercept),0.03033314,0.0138957,0.06621464
echo1,0.62103869,0.4649972,0.82944374
age,1.02238142,1.0129557,1.03189486
vent1,2.09362509,1.2247994,3.57876229
sedative1,1.06282538,0.6573555,1.71839715
icd_chf1,0.82393269,0.5904508,1.14974034
icd_afib1,1.24525663,0.9103403,1.70338947
icd_cad1,0.70252835,0.4700399,1.05000884
icd_stroke1,1.62161389,0.8765702,2.99990998
vs_cvp_first,1.00241137,0.9961913,1.00867025
